In [1]:
import pyodbc as p
import pandas as pd

conn = p.connect('DSN=myDSN;UID=MyID;PWD=MyPassword')

report_date='31.01.2019'
author='Martin Vavpotič'

year_list = ['0','january','february','march','april','may','june','july','august','september','october','november','december']
# Probably a better way to do this; I'm sure there are libraries for this but I'm still learning how to handle lists.

Extra_Life = 55_000
Extra_Medical = 75_000

sql = "select a.product_name,round(sum(a.alfa),2) alfa from martinv.table1 a, martinv.table2 b "
sql+= "where a.report_date = to_date(' " +report_date+ " ','dd.mm.yyyy') "
sql+= "and a.pk_prod=b.pk_prod "
sql+= "and a.pk_ins=b.pk_ins "
sql+= "and b.alfa_bonus=1 "
sql+= "and (a.status='V-VEL' or (a.status='S-PRE' and a.last_storno_req_valid_from > to_date(' " +report_date+ " ','dd.mm.yyyy')) "
sql+= "or a.status_code='V-VEL') "
sql+= "and a.alfa > 0 "
sql+= "group by a.product_name "
sql+= "order by 1;"

alfa_cost = pd.read_sql(sql,conn)
alfa_cost = alfa_cost.set_index('PRODUCT_NAME')['ALFA'].to_dict() # alfa_cost is turned from a N×2 dataframe into a dict;
# first column turns into keys that are products, second column is sumation of alfa costs over products, these are turned to values of dict.

alfa_modified = dict(alfa_cost)
#Copies alfa_cost dict for future work. By declaring it a dict alfa_modified is a copy of alfa_cost, not its reference keyword.

prod_02 = ('ProductLife08','ProductLife11','ProductLife12')
# A list of product that has to be treated in a special way.

# Turn a tuple prod_02 into a string with parenthesis and commas in between.
# The tuple can lengthen later, this loop handles it the same no matter how many products are added to the list.
prod_vec = "('"
for prod in prod_02:
    prod_vec += prod + "','"
prod_vec = prod_vec[0: len(prod_vec)-2] + ")"

# Second query, specified by additional parameter: ben_code. This is only done for a subset of products as well.
sql_02 = "select product_name, round(sum(alfa),2) alfa_02 from martinv.table1 "
sql_02+= "where ben_code like '02.%' and product_name in "+prod_vec+" 
sql_02+= "and report_per=to_date(' " +report_date+ " ','dd.mm.yyyy') "
sql_02+= "and alfa >0 "
sql_02+= "group by product_name "
sql_02+= "order by 1;"

alfa_02 = pd.read_sql(sql_02, conn)
alfa_02 = alfa_02.set_index('PRODUCT_NAME')['ALFA_02'].to_dict()

conn.close()

# Subtract from the first dict what is included in the second dict, do it only if products match.
for prod in prod_02:
    alfa_modified[prod] = round(alfa[prod] - alfa_02[prod],2)

# Adding to the first dict elements of the second dict. Modifying its keys first so they don't mix with the existing ones, otherwise...
# the original dict would simply have updated values.
for prod in list(alfa_02.keys()):
    alfa_modified.update({prod+'_CI':alfa_02[prod]})

# I now know it's not needed to turn keys of a dict into a list to be able to loop over them but I kept the original code to show my current 
# ...knowledge level.
    
# Summing up all alfa costs for a. Using a for loop with list comprehension.
# I'm aware two ifs in the list comprehension make it unclear and that this shouldn't be done but I was still experimenting with it...
# ...and I wanted to see how this would work out.
alfa_Life_sum = round(sum([alfa_modified[prod] for prod in list(alfa_modified.keys()) if 'Life' in prod if 'CI' not in prod]),2)

# The proper way to do it (more readable code):
#for prod in list(alfa_m.keys()):
#    if 'Life' in prod:
#        if 'CI' not in prod:
#            alfa_Life_sum = round(sum(alfa_m[prod]),2)

# The other sumation, based on which product have the KB tag.
alfa_Med_sum = round(sum([alfa_modified[prod] for prod in list(alfa_modified.keys()) if 'Life' in prod if 'CI' in prod]),2)

# I'm using two naming conventions, one with _, one without it. Experimenting to see which one I prefer, would usually use just one.
LifeExtra = {}
MedicalExtra = {}

# Another way of filling a dict without the update method as before. More clearly stated, more intuitive.
# Dividing the previously defined Extra costs based on the relative portion of costs for each product.
for prod in list(alfa_modified.keys()):
    if 'Life' in prod:
        if 'CI' in prod:
            MedicalExtra[prod] = round(alfa_modified[prod]/alfa_Med_sum *Extra_Medical,2)
        else:
            LifeExtra[prod] = round(alfa_modified[prod]/alfa_Life_sum *Extra_Life,2)

In [19]:
alfa_Life_sum_m = []

# Making sure that I get the same result if I use .append and if use should use elif or is else specific enough
# (might change in the future)
for prod in alfa_m.keys():
    if 'Life' in prod:
        if 'CI' not in prod:
            alfa_Life_sum_m.append(alfa_modified[prod])

In [6]:
# Writing the produced numbers into an Excel file. I have an empty template Excel file.

import openpyxl as op

my_file = op.load_workbook(filename='template.xlsx', read_only=False) #workbook
wsheet = my_file.active #worksheet

# 1. Look for the first line and column; searching for the name of the first product on the list.
for row in range(1, wsheet.max_row):
    for col in range(1, wsheet.max_column):
        if wsheet.cell(row, col).value == 'InvProduct01':
            row_i = row
            col_i = col
            break
    else: # this else is attached to the for loop. It works to exit the nested for loops (think of it as a no-break statement).
        continue
    break

# 2. Searching for the empty line after all the product have been listed:
for row in range(row_i, wsheet.max_row):
    if wsheet.cell(row, col_i).value is None:
        row_f = row
        break

# 3. Position where signature and date need to be placed:
for row in range(row_i,ws.max_row):
    if 'Prepared by' in str(ws.cell(row,col_i).value):
        row_sig = row
        break
        
        
# 4. Reads the products from the template file, writes down results of the data query one column to the right, writes extra costs further
# to the right. Try/except is used in case the query doesn't contain the key, specified by the template file. This would make sense if the
# costs were stuck to 0 for whatever reason.
for row in range(row_i, row_f):
    try:
        wsheet.cell(row, col_i +1).value = alfa_modified[wsheet.cell(row, col_i).value]
    except KeyError:
        wsheet.cell(row, col_i +1).value = 0
    
    try:
        wsheet.cell(row, col_i +2).value = LifeExtra[wsheet.cell(row, col_i).value]
    except KeyError:
        wsheet.cell(row, col_i +2).value = 0
        
    try:
        wsheet.cell(row, col_i +3).value = MedicalExtra[wsheet.cell(row, col_i).value]
    except KeyError:
        wsheet.cell(row, col_i +3).value = 0

        
# 5. Updates author and date; usually done by me; this can be changed either in the code or in the template Excel file itself.
wsheet.cell(row_s +1, col_i).value = author
wsheet.cell(row_s +2, col_i).value = dt.now().strftime('%d.%m.%Y')


# 6. Saves the file with a new name; date is taken from the date string, used in the sql query.
fajl.save('Transfer alfa costs_'+year_list[int(report_date[3:5])]+'_accountancy_temp.xlsx')